In [42]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

In [43]:
data = load_diabetes()

In [44]:
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

In [45]:
X = data.data
feature_names = data.feature_names
y = data.target
print(len(X))

442


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=60)
Scaler = StandardScaler()
X_train = Scaler.fit_transform(X_train)
X_test = Scaler.transform(X_test)

In [47]:
model = Sequential([
    Dense(300, activation = 'relu'),
    Dropout(0.4),
    Dense(300, activation = 'relu'),
    Dropout(0.4),
    Dense(300, activation = 'relu'),
    Dropout(0.4),
    Dense(1),
])

In [48]:
model.compile(optimizer='Nadam',
    loss='mse',
    metrics=['mae']
)

In [49]:
callback = ModelCheckpoint('best_model.keras', monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=2000, validation_split=0.3, callbacks=[callback])

Epoch 1/2000
6/8 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 25754.3711 - mae: 142.5753 
Epoch 1: val_loss improved from inf to 33217.54688, saving model to best_model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 52s 710ms/step - loss: 26209.7559 - mae: 144.0010 - val_loss: 33217.5469 - val_mae: 163.9061
Epoch 2/2000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 26587.9004 - mae: 145.2439
Epoch 2: val_loss improved from 33217.54688 to 28970.83789, saving model to best_model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - loss: 26469.3438 - mae: 144.8007 - val_loss: 28970.8379 - val_mae: 150.8508
Epoch 3/2000
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 19727.0293 - mae: 121.4689
Epoch 3: val_loss improved from 28970.83789 to 16770.51172, saving model to best_model.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 19770.9922 - mae: 121.0452 - val_loss: 16770.5117 - val_mae: 107.4439
Epoch 4/2000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 11388.5625 - mae: 83.9906
Epoch 4: val_loss improved from 16

In [51]:
model = load_model('best_model.keras')
test = model.evaluate(X_test, y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - loss: 3458.9460 - mae: 45.5818


In [52]:
print(y_test[10])

214.0


In [53]:
model.predict(X_test[10].reshape(1, -1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[150.80374]], dtype=float32)

In [54]:
base_learners = [
    ('rf', RandomForestRegressor(n_estimators=200, random_state=60)),
    ('svr', SVR()),
    ('lr', LinearRegression())
]



In [55]:
blender = MLPRegressor(
    hidden_layer_sizes=(300, 300, 200, 100),
    activation='relu',
    max_iter=2000,
    random_state=60,
    solver='adam',
    alpha=0.001,
    learning_rate='adaptive',
    batch_size='auto',
    tol=1e-4,
)
stacking_model = StackingRegressor(estimators=base_learners, final_estimator=blender)

stacking_model.fit(X_train, y_train)

StackingRegressor(estimators=[('rf',
                               RandomForestRegressor(n_estimators=200,
                                                     random_state=60)),
                              ('svr', SVR()), ('lr', LinearRegression())],
                  final_estimator=MLPRegressor(alpha=0.001,
                                               hidden_layer_sizes=(300, 300,
                                                                   200, 100),
                                               learning_rate='adaptive',
                                               max_iter=2000, random_state=60))

In [56]:
score = stacking_model.score(X_test, y_test)
print(f"R² do modelo de Stacking: {score:.4f}")


R² do modelo de Stacking: 0.5252


In [58]:
predictions = stacking_model.predict(X_test)

print(f"Primeiras 10 previsões: {predictions[:10]}")
print(f"Primeiros 10 alvos: {y_test[:10]}")

Primeiras 10 previsões: [183.09882465 161.93339352 133.79419447  84.9921725   73.01706764
 235.24734603 189.05029264 248.48988353 227.75911109 150.98160836]
Primeiros 10 alvos: [122.  90. 214.  84.  72. 280. 171. 128. 332. 142.]
